In [1]:
#Importing selenium necessary packages, here i'm working with chrome driver
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# BeautifulSoup for parsing HTML
from bs4 import BeautifulSoup as soup

import requests
import time
import pymongo

In [2]:
#city = "djelfa" chosen because it's nearly the center of Algeria, and it's the best option to scrap data with a big radius !
base_url = "https://web.facebook.com/marketplace/constantine/propertyforsale/?"

#Djelfa code in url = 112237105459123

sorting = "price_descend"

#You can add another criteria to the url depending on your needs
minPrice = 200
#maxPrice = ?
#longitude = ?
#latitude = ?

#consider changing the url as your needs
url = f"{base_url}sortBy={sorting}&minPrice={minPrice}&radius=25"

In [6]:
# Working with selenium headless browser
options = webdriver.ChromeOptions() 
options.headless = True

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options) 
driver.get(url)

source = "Facebook MarketPlace",

# Function that stores the announcements in the data base
def save_to_database(records):
    client = pymongo.MongoClient('mongodb://localhost:27017')
    mydb = client["Real-Estate"]
    information = mydb.RealEstateListing
    if information is not None:
        existing_urls = []
        for record in information.find():
            existing_urls.append(record['Link'])
                
        for record in records:
            if record['Link'] in existing_urls:
                continue
            else:
                information.insert_one(record)
    
    else:
        information.insert_many(records)

# A function that closes the Facebook popup login windows
def close_login_page():
    try:
        popup_close_button = driver.find_element(By.XPATH, "//div[@class='x92rtbv x10l6tqk x1tk7jg1 x1vjfegm']")
        popup_close_button.click()

        # Wait for the popup windows to close
        time.sleep(2)
    except NoSuchElementException:
        pass

def next_image():
    try:
        next_image_button = driver.find_element(By.XPATH, "//div[@aria-label='View next image']")
        next_image_button.click()

        # Wait for the image to be shown
        time.sleep(2)

        # Check if the new image source is different from the first one
        new_image_source = driver.find_element(By.XPATH, "//img").get_attribute("src")
        return new_image_source
    except NoSuchElementException:
        return None

# A function to scroll to the bottom of the page
def scroll_to_bottom(driver):
    close_login_page()
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(6)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


# Scroll to the bottom of the page to load all content
scroll_to_bottom(driver)

# Waiting for all the elements to be visible
wait = WebDriverWait(driver, 50)
parent_elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//a[@class='x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv']")))

# Scraping data from each element and putting them in a list
listings_data = [] 

toBeRemoved = ["[hidden information]", "See less"]

# Collecting posts links firstly
parent_links = [parent_element.get_attribute("href") for parent_element in parent_elements]
# Iterate over links to visit each page separately

try:
    
    for listing_link in parent_links: 
        driver.get(listing_link)
        time.sleep(3)
    
        close_login_page()
        
        try:
            driver.execute_script("window.scrollBy(0, 300);")
            time.sleep(1)
            
            listing_description_element = driver.find_element(By.XPATH, ".//div[@class='xz9dl7a x4uap5 xsag5q8 xkhd6sd x126k92a']")
            see_more_button = None
            
            try:
                see_more_button = listing_description_element.find_element(By.XPATH, ".//span[contains(text(), 'See more')]")
            except NoSuchElementException:
                pass
            
            if see_more_button:
                see_more_button.click()
                time.sleep(3)
    
            listing_description = listing_description_element.text.strip()
            
            if not listing_description:
                listing_description = "Description not available"
    
            for word in toBeRemoved:
                listing_description = listing_description.replace(word, "")

            listing_price = driver.find_element(By.XPATH, ".//span[@class='x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u']").text
            listing_title = driver.find_element(By.XPATH, ".//span[@class='x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x14z4hjw x3x7a5m xngnso2 x1qb5hxa x1xlr1w8 xzsf02u']").text
            
            for word in toBeRemoved:
                listing_title = listing_title.replace(word, "")
            
            listing_location = driver.find_element(By.XPATH, ".//span[@class='x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u x1yc453h']").text 
            listing_image_elements = driver.find_elements(By.XPATH, ".//img")
            images_data = []
            first_image_source = listing_image_elements[0].get_attribute("src")
            
            images_data.append(first_image_source)  # Append the first image source
    
            while True:
                new_image_source = next_image()
                if new_image_source and new_image_source != first_image_source:
                    images_data.append(new_image_source)
                else:
                    break
            
        except Exception as e:
            print("Error:", e)
        
        temporary_listings_data = { 
            "Title": listing_title,
            "Price": listing_price,
            "Location": listing_location,
            "Description": listing_description,
            "Images": images_data,
            "Source": source,
            "Date": None,
            "Link": listing_link,
            "Category": None,
            "Surface": None,
        } 
        listings_data.append(temporary_listings_data) 
        print(temporary_listings_data)
        
finally:
    save_to_database(listings_data)
    print("Scraped items: ", listings_data)




driver.quit()



{'Title': '0553225784 0779327032', 'Price': 'DZD550,000,000', 'Location': 'قسنطينة', 'Description': 'وكالة عقارية تعرض للبيع شقة اف3 عقد معلق بشرط وج18 ترقية ناصري علي منجلي طابق 2 قريبة لجميع متطلبات الحياة مدارس ومحلات تجارية المهتم يتصل بي الوكالة شكرا لا أرد على تعليقات ', 'Images': ['https://scontent.falg7-1.fna.fbcdn.net/v/t39.30808-6/437954436_800495908621489_1735213251459577480_n.jpg?_nc_cat=109&ccb=1-7&_nc_sid=5f2048&_nc_ohc=FRUq5ErGr3YAb7VWYG6&_nc_ht=scontent.falg7-1.fna&oh=00_AfBmvuLYfq1DFxSh1PcpO5hU4Cand9lztbA5r4XZTxOzKg&oe=662EDD0C', 'https://scontent.falg7-1.fna.fbcdn.net/v/t39.30808-6/438681100_800493151955098_5592792611164413649_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=5f2048&_nc_ohc=lnfJEoqPq4sAb7Ys8qx&_nc_ht=scontent.falg7-1.fna&oh=00_AfBtTu5GRkm9JoIT_AUOxoA0EUXYk6EGkPRHfy-Tz2OYGQ&oe=662EEA65', 'https://scontent.falg7-5.fna.fbcdn.net/v/t39.30808-6/435664398_800492938621786_7100292013672063281_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=5f2048&_nc_ohc=5l7nLtfceFQAb7yCJuh&_nc_ht=sconten

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.61)
Stacktrace:
	GetHandleVerifier [0x00FBC113+48259]
	(No symbol) [0x00F4CA41]
	(No symbol) [0x00E40A17]
	(No symbol) [0x00E1E02B]
	(No symbol) [0x00EA742E]
	(No symbol) [0x00EB9476]
	(No symbol) [0x00EA0B36]
	(No symbol) [0x00E7570D]
	(No symbol) [0x00E762CD]
	GetHandleVerifier [0x01276533+2908323]
	GetHandleVerifier [0x012B3B4B+3159739]
	GetHandleVerifier [0x0105505B+674763]
	GetHandleVerifier [0x0105B21C+699788]
	(No symbol) [0x00F56244]
	(No symbol) [0x00F52298]
	(No symbol) [0x00F5242C]
	(No symbol) [0x00F44BB0]
	BaseThreadInitThunk [0x76247BA9+25]
	RtlInitializeExceptionChain [0x772CBE3B+107]
	RtlClearBits [0x772CBDBF+191]
